In [1]:
%pip install gensim sklearn_crfsuite pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 24.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import sklearn_crfsuite
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


In [4]:
# Cargar el dataset de SQL Injection
df = pd.read_csv("/data/SQLiV3.csv")  # El dataset debe tener columnas 'Sentence' y 'Label'
print(df.head())
data = pd.read_csv('/home/gbenito/universidad/TP-Final-BDM/data/SQLiV3.csv')
data.columns = ['Sentence', 'Label']
data['Sentence'] = data['Sentence'].astype(str)


                                            Sentence Label Unnamed: 2  \
0                  " or pg_sleep  (  __TIME__  )  --     1        NaN   
1  create user name identified by pass123 tempora...   NaN          1   
2   AND 1  =  utl_inaddr.get_host_address   (    ...     1        NaN   
3   select * from users where id  =  '1' or @ @1 ...     1        NaN   
4   select * from users where id  =  1 or 1#"  ( ...     1        NaN   

   Unnamed: 3  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  


In [ ]:
# Preprocesamiento de textos
data['Tokens'] = data['Sentence'].apply(word_tokenize)
sentences = data['Tokens'].tolist()


In [ ]:
# Generación de embeddings con Word2Vec
embedding_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
embeddings = {word: embedding_model.wv[word] for word in embedding_model.wv.index_to_key}


In [ ]:
# Convertir las oraciones en secuencias de embeddings
def sentence_to_embeddings(sentence):
    return [embeddings[word] for word in sentence if word in embeddings]

data['Embeddings'] = data['Tokens'].apply(sentence_to_embeddings)


In [ ]:
# Preparar los datos para CRF
X = data['Embeddings'].tolist()
y = data['Label'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Entrenar el modelo CRF
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', max_iterations=100)
crf.fit(X_train, y_train)


In [ ]:
# Evaluar el modelo
y_pred = crf.predict(X_test)
print(sklearn_crfsuite.metrics.flat_classification_report(y_test, y_pred))
